In [ ]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that translates {input_language} to {output_language}. Translate the following sentence: {sentence}"), 
        ("human", "{sentence}")
    ]
)

print(prompt.format(input_language="English", output_language="French", sentence="Hello, how are you?"))

model = ChatOllama(model="llama3.2", temperature=0.7)

chain = prompt | model | StrOutputParser()

print(chain.invoke({"input_language": "English", "output_language": "French", "sentence": "Hello, how are you?"}))

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can suggest some {product} company names. Please provide the names in a comma-separated format."),
    ("human", "Suggest some company names for a company that makes {product}.")
])

chain = prompt | model | CommaSeparatedListOutputParser()

try:
    print(chain.invoke({"product": "smartphones"}))
except Exception as e:
    print(e)

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class CompanyName(BaseModel):
    names: list[str] = Field(..., description="A list of company names in a comma-separated format.")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can suggest some {product} company names. Please provide the names in {format_instruction}."),
    ("human", "Suggest some company names for a company that makes {product}.")
])

parser = JsonOutputParser(pydantic_object=CompanyName)

chain = prompt | model | JsonOutputParser(pydantic_object=CompanyName)

try:
    print(chain.invoke({"product": "smartphones", "format_instruction": parser.get_format_instructions()}))
except Exception as e:
    print(e)

In [ ]:
from langchain_core.documents import Document

document_1 = Document(page_content="Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'")

document_2 = Document(page_content="So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.")

prompt = ChatPromptTemplate.from_template("Summarize the following documents: {documents}")

chain = prompt | model | StrOutputParser()

try:
    print(chain.invoke({"documents": [document_1, document_2]}))
except Exception as e:
    print(e)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


loader = WebBaseLoader("https://en.wikipedia.org/wiki/LangChain")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = Chroma.from_documents(splits, embeddings)
retriever = vector_store.as_retriever()

def format_documents(documents: list[Document]) -> str:
    return "\n\n".join([document.page_content for document in documents])

prompt = ChatPromptTemplate.from_template("Answer based ONLY on context: {context}\nQuestion: {question}")

rag_chain = (
    {
    "context": lambda x: format_documents(retriever.invoke(x["question"])),
    "question": lambda x: x["question"]
    } 
    | prompt 
    | model 
    | StrOutputParser()
)

try:
    print(rag_chain.invoke({"question": "What is LangChain?"}))
except Exception as e:
    print(e)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import MessagesPlaceholder


loader = WebBaseLoader("https://en.wikipedia.org/wiki/LangChain")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = Chroma.from_documents(splits, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

def format_documents(documents: list[Document]) -> str:
    return "\n\n".join([document.page_content for document in documents])

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based on the provided {context}."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

rag_chain = (
    {
    "context": lambda x: format_documents(retriever.invoke(x["question"])),
    "chat_history": lambda x: x["chat_history"],
    "question": lambda x: x["question"], 
    } 
    | prompt 
    | model 
    | StrOutputParser()
)

chat_history = []

try:
    while True:
        _input = input("Ask a question (or 'exit' to quit): ")
        if _input.lower() == "exit":
            break
        print(f"You: {_input}\n")
        response = rag_chain.invoke({
            "question": _input,
            "chat_history": chat_history
        })
        
        chat_history.append(HumanMessage(content=_input))
        chat_history.append(AIMessage(content=response))

        print(f"AI: {response}\n")
except Exception as e:
    print(e)